# 1. Setup e carregamento de dados

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
from IPython.display import display

# Configurar o estilo visual para os gráficos
sns.set_style("whitegrid")

# Caminho para o ficheiro de dados processados
caminho_dados = '../data/dados_processados_trimestrais.csv'

try:
    # Carregar o DataFrame, usando a primeira coluna (índice 0) como índice temporal
    df_final = pd.read_csv(caminho_dados, index_col=0, parse_dates=True)
    
    # Atribuir o nome 'data' ao índice para clareza e consistência
    df_final.index.name = 'data'
    
    # Renomear as colunas para nomes mais curtos e fáceis de usar
    df_final.rename(columns={
        'PIB_var_homologa': 'pib',
        'Credito_Empresas_Total': 'credito_empresas',
        'Credito_Particulares_Total': 'credito_particulares',
        'Endividamento_Total': 'endividamento'
    }, inplace=True)

    print("DataFrame carregado e preparado com sucesso!")
    display(df_final.head())

except FileNotFoundError:
    print(f"ERRO: O ficheiro de dados não foi encontrado em '{caminho_dados}'. Verifique o caminho.")

DataFrame carregado e preparado com sucesso!


,pib,credito_empresas,credito_particulares,endividamento
data,,,,
2007-12-31,3.0,4877.6,11.6,156083.31
2008-03-31,1.7,5278.5,11.8,158296.02
2008-06-30,0.9,6077.3,13.0,160952.95
2008-09-30,0.5,6898.4,14.1,162444.83
2008-12-31,-1.9,7738.9,14.7,163756.38


# 2. Centralizar resultados das anomalias

In [3]:
# Datas de anomalias detetadas pelo IsolationForest (confirmadas)
anomalias_isoforest_datas = [
    '2012-06-30', '2012-09-30', '2012-12-31', # Crise da Dívida Soberana
    '2020-06-30',                             # Choque COVID-19
    '2021-06-30',                             # Recuperação Pós-COVID (efeito base)
    '2022-03-31'                              # Início Crise Inflacionária / Guerra da Ucrânia
]

# Datas de anomalias detetadas pela Decomposição STL (confirmadas, agregadas das 4 séries)
anomalias_stl_datas = [
    '2020-06-30', '2021-06-30',               # Do PIB
    '2011-09-30',                             # Do Crédito a Empresas
    '2017-03-31', '2017-06-30',               # Do Crédito a Particulares
    '2018-12-31', '2019-03-31'                # Do Endividamento
]

# Datas de anomalias detetadas pelo Prophet (confirmadas no PIB)
anomalias_prophet_datas = [
    '2020-06-30', # Queda abrupta (COVID)
    '2021-06-30', # Recuperação muito forte (Efeito Base)
    '2021-12-31', # Pico de recuperação (superior à previsão)
    '2022-09-30'  # Crescimento forte inesperado (superior à previsão)
]

# Converter as listas de strings para objetos DatetimeIndex para correspondência segura
anomalias_isoforest_idx = pd.to_datetime(anomalias_isoforest_datas)
anomalias_stl_idx = pd.to_datetime(anomalias_stl_datas)
anomalias_prophet_idx = pd.to_datetime(anomalias_prophet_datas)

# Inicializar as colunas de anomalias com 0 (não anomalia)
df_final['anomalia_isoforest'] = 0
df_final['anomalia_stl'] = 0
df_final['anomalia_prophet'] = 0

# Marcar com 1 (anomalia) as datas correspondentes
df_final.loc[df_final.index.isin(anomalias_isoforest_idx), 'anomalia_isoforest'] = 1
df_final.loc[df_final.index.isin(anomalias_stl_idx), 'anomalia_stl'] = 1
df_final.loc[df_final.index.isin(anomalias_prophet_idx), 'anomalia_prophet'] = 1

print("Colunas de anomalias adicionadas ao DataFrame.")
print("\nVerificação para confirmar que as colunas foram criadas:")
display(df_final.loc['2020':'2021', ['pib', 'anomalia_isoforest', 'anomalia_stl', 'anomalia_prophet']])

Colunas de anomalias adicionadas ao DataFrame.

Verificação para confirmar que as colunas foram criadas:


,pib,anomalia_isoforest,anomalia_stl,anomalia_prophet
data,,,,
2020-03-31,-2.4,0,0,0
2020-06-30,-17.6,1,1,1
2020-09-30,-6.1,0,0,0
2020-12-31,-6.6,0,0,0
2021-03-31,-5.4,0,0,0
2021-06-30,16.5,1,1,1
2021-09-30,5.5,0,0,0
2021-12-31,7.2,0,0,1


In [4]:
# Criar a coluna 'contagem_anomalias' somando o número de modelos que detetaram uma anomalia
df_final['contagem_anomalias'] = df_final[['anomalia_isoforest', 'anomalia_stl', 'anomalia_prophet']].sum(axis=1)

# Criar o DataFrame 'df_comparativo' contendo apenas as linhas com pelo menos uma anomalia
df_comparativo = df_final[df_final['contagem_anomalias'] > 0].copy()

# Ordenar a tabela: primeiro por maior consenso (mais importante), depois por data
df_comparativo.sort_values(by=['contagem_anomalias', 'data'], ascending=[False, True], inplace=True)

# Selecionar e reordenar as colunas para uma apresentação final clara
colunas_para_exibir = [
    'pib', 'credito_empresas', 'credito_particulares', 'endividamento',
    'anomalia_isoforest', 'anomalia_stl', 'anomalia_prophet', 'contagem_anomalias'
]
df_comparativo = df_comparativo[colunas_para_exibir]

print("Tabela Comparativa de Anomalias gerada com sucesso!")
print("\nEsta tabela resume todos os eventos anómalos detetados pelos 3 modelos:")
display(df_comparativo)

Tabela Comparativa de Anomalias gerada com sucesso!

Esta tabela resume todos os eventos anómalos detetados pelos 3 modelos:


,pib,credito_empresas,credito_particulares,endividamento,anomalia_isoforest,anomalia_stl,anomalia_prophet,contagem_anomalias
data,,,,,,,,
2020-06-30,-17.6,8896.8,18.8,143217.50,1,1,1,3
2021-06-30,16.5,7105.6,16.5,147519.12,1,1,1,3
2011-09-30,-2.3,19208.8,27.0,165525.90,0,1,0,1
2012-06-30,-4.0,27934.1,32.1,160264.04,1,0,0,1
2012-09-30,-4.3,31060.9,33.5,158446.81,1,0,0,1
2012-12-31,-4.6,31471.3,34.7,156432.13,1,0,0,1
2017-03-31,3.7,34215.1,31.0,135192.51,0,1,0,1
2017-06-30,3.5,32723.8,37.6,134992.61,0,1,0,1
2018-12-31,2.9,19948.5,26.9,136294.43,0,1,0,1


# 3. Visualização Interativa do PIB com Todas as Anomalias

In [5]:
import plotly.graph_objects as go

# Criar a figura principal
fig = go.Figure()

# 1. Adicionar a linha da série temporal do PIB
fig.add_trace(go.Scatter(
    x=df_final.index,
    y=df_final['pib'],
    mode='lines',
    name='Variação do PIB (%)',
    line=dict(color='lightgrey')
))

# 2. Adicionar marcadores para as anomalias do Isolation Forest
df_anom_iso = df_final[df_final['anomalia_isoforest'] == 1]
fig.add_trace(go.Scatter(
    x=df_anom_iso.index,
    y=df_anom_iso['pib'],
    mode='markers',
    name='Anomalia: Isolation Forest (Sistémica)',
    marker=dict(color='red', size=12, symbol='circle')
))

# 3. Adicionar marcadores para as anomalias do STL
df_anom_stl = df_final[df_final['anomalia_stl'] == 1]
fig.add_trace(go.Scatter(
    x=df_anom_stl.index,
    y=df_anom_stl['pib'],
    mode='markers',
    name='Anomalia: STL (Ponto de Viragem)',
    marker=dict(color='green', size=12, symbol='diamond')
))

# 4. Adicionar marcadores para as anomalias do Prophet
df_anom_prophet = df_final[df_final['anomalia_prophet'] == 1]
fig.add_trace(go.Scatter(
    x=df_anom_prophet.index,
    y=df_anom_prophet['pib'],
    mode='markers',
    name='Anomalia: Prophet (Desvio da Previsão)',
    marker=dict(color='purple', size=12, symbol='x')
))

# 5. Adicionar marcadores especiais para os consensos (onde a contagem > 1)
df_consenso = df_final[df_final['contagem_anomalias'] > 1]
fig.add_trace(go.Scatter(
    x=df_consenso.index,
    y=df_consenso['pib'],
    mode='markers',
    name='Consenso (Múltiplos Modelos)',
    marker=dict(color='gold', size=18, symbol='star', line=dict(color='black', width=1))
))


# 6. Melhorar o Layout
fig.update_layout(
    title='Análise Comparativa de Anomalias no PIB de Portugal',
    xaxis_title='Data',
    yaxis_title='Variação Homóloga do PIB (%)',
    legend_title='Legenda',
    template='plotly_white'
)

# Exibir o gráfico interativo
fig.show()

# 4. Análise Final e Conclusões

A visualização comparativa dos resultados dos três modelos — Isolation Forest, Decomposição STL e Prophet — permite extrair insights valiosos sobre a natureza das anomalias macroeconómicas em Portugal.

### 1. Consenso nos Eventos Extremos (Anomalias de Contagem 3)

Os pontos onde todos os modelos concordam (marcados com uma **estrela dourada**) representam os choques mais violentos e inequívocos na economia portuguesa recente:
*   **2º Trimestre de 2020 (Q2 2020):** A queda abrupta do PIB devido ao primeiro confinamento da COVID-19. Este evento foi tão extremo que foi capturado como uma anomalia sistémica (Isolation Forest), um ponto de viragem drástico (STL) e um desvio massivo da previsão (Prophet).
*   **2º Trimestre de 2021 (Q2 2021):** O pico da recuperação pós-pandemia. O crescimento excecionalmente alto, impulsionado pelo "efeito base" estatístico, foi igualmente identificado por todos os modelos como um evento fora do normal.

### 2. A Especialização e Complementaridade dos Modelos

A verdadeira força desta análise reside nos pontos onde os modelos divergem, pois cada um revela um tipo diferente de anomalia:

*   **Isolation Forest (Círculos Vermelhos):** Este modelo provou ser um excelente detetor de **desequilíbrios sistémicos**. As anomalias que ele detetou isoladamente, como durante a **crise da dívida soberana em 2012** ou no **início da crise inflacionária em 2022**, representam períodos em que a *relação* entre o PIB, o crédito e o endividamento estava fora do padrão histórico, mesmo que uma única variável não estivesse no seu máximo ou mínimo histórico.

*   **Decomposição STL (Losangos Verdes):** Este modelo destacou-se na identificação de **pontos de viragem** em variáveis individuais. As anomalias que ele encontrou em **2011, 2017 e 2018/2019** correspondem a momentos cruciais como o início da contração do crédito (*credit crunch*) e os mínimos do ciclo de endividamento, eventos que alteraram a tendência de uma série específica.

*   **Prophet ("X" Roxo):** Focado exclusivamente no PIB, este modelo funcionou como um "vigia de expectativas". As anomalias que detetou em **finais de 2021 e meados de 2022** não foram necessariamente as de maior ou menor valor absoluto, mas sim aquelas em que o crescimento real da economia **superou significativamente a previsão** do modelo, indicando uma resiliência ou aceleração inesperada.

### Conclusão Final

Nenhum modelo isolado teria contado a história completa. A combinação de uma abordagem multivariada (Isolation Forest), univariada (STL) e preditiva (Prophet) fornece uma visão de 360 graus das perturbações económicas. Conseguimos não só identificar os grandes choques, mas também diferenciar entre desequilíbrios estruturais, pontos de viragem de tendência e surpresas no crescimento.

---

**Próximo Passo Final: Construir o Dashboard Interativo**

Com a análise e os artefactos visuais concluídos, o último passo do projeto é levar estes elementos para um dashboard interativo (usando Streamlit ou Dash) que permita a um utilizador final explorar estes resultados de forma dinâmica.